In [1]:
import operator
import math
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
#函数：x^4+x^3+x^2+x

In [8]:
#自定义一个除法函数，是为了避免出现除0异常
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

#下面是定义的原语集，也就是符号集和常量
pset = gp.PrimitiveSet("MAIN", 1)  #一个变量
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1) 
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))  #插入一个特殊的常量集，供叶节点选择。 值是从-1,0,1中的一个
pset.renameArguments(ARG0='x')   #就是给变量改个名，不改也行，原名就是ARG0

#适应度和个体的基因型
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))   #单目标，最小化
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)  #因为是基于树，所以用PrimitiveTree原始集，后面就表示适应度了

#设置toolbox
toolbox = base.Toolbox()  #创建一个工具箱实例
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)  #通过halfandhalf的方法生成，最小深度和最大深度也要给出
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr) #个体生成的方式。两个参数分别是个体类型（即基因型如何存储）和生成方法
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  #种群的生成方式。两个参数分别是种群的存储方式，个体生成的方式。
toolbox.register("compile", gp.compile, pset=pset)  #编译表达式

#将符号树转换为算式的形式并计算误差
def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),       #DEAP将适应度存储为可迭代对象，所以必须返回一个元组

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])  #自定义的评估函数。point可理解为是样例集合
toolbox.register("select", tools.selTournament, tournsize=3) #选择
toolbox.register("mate", gp.cxOnePoint) #交叉方法
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2) #生成突变子树
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset) #随机选一个节点，用生成的子树替代它

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))  #用交叉和突变的方式限制树的高度，避免膨胀。17是个经验值
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

#这些类和实例都声明过了，再运行一次就会报错

Exception: Ephemerals with different functions should be named differently, even between psets.

In [6]:
def main():
    random.seed(318)
    
    #生成初始种群，设置一些参数信息
    pop = toolbox.population(n=300) #生成初始种群
    hof = tools.HallOfFame(1) #保留从演化开始以来的较好的个体，1表示只保留最好的一个

    #展示一些统计数据
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)  #适应度
    stats_size = tools.Statistics(len) #每个个体的大小。具体怎么衡量的还没查阅
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)  #多种统计信息，即适应度和大小
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,  #40是演化代数
                                   halloffame=hof, verbose=True)  #调用演化算法，verbose表示记录演化过程中的信息
    # print log
    return pop, log, hof

if __name__ == "__main__":
    main()

   	      	                        fitness                        	                      size                     
   	      	-------------------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max  	min     	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	1.78879	0  	30.34	0.450825	300   	2.67896	3.54667	0  	7  	2  	300   	1.49482
1  	166   	1.43254	1  	44.4437	0.183711	166   	3.05668	3.60667	1  	12 	1  	166   	1.77725
2  	166   	2.16879	2  	315.736	0.165572	166   	18.1873	3.55   	2  	9  	1  	166   	1.62506
3  	163   	0.98255	3  	2.9829 	0.165572	163   	0.712666	3.42667	3  	9  	1  	163   	1.45073
4  	153   	0.836017	4  	14.538 	0.165572	153   	0.979399	3.77   	4  	11 	1  	153   	1.64025
5  	158   	0.944635	5  	18.9739	0.165572	158   	1.61614 	3.77667	5  	10 	1  	158   	1.62894
6  	169   	0.885819	6  	14.2181	0.165572	169   	1.00296 	4      	6  	10 	1  	169   	1.87617
7  	167   	0.731332	7  	3.35292	0.165572	167   